In [2]:
import math 
import pandas as pd
import random 
import numpy as np
import pickle

In [3]:
# Setting simulation parameters values

N = [30, 50, 75, 100, 200, 300, 500, 1000]  # population size
D = [0.01, 0.1, 0.2, 0.5, 0.75, 1, 1.5, 2] # Densities
T = [1, 2, 3, 4, 5, 7, 10] # infectious time in days

d = 2 # distance in meters 
iter = 100 # number of iterations
time = 1440 # exposure time in minutes
T_max = 10


In [6]:
# importing trace

with open(r"D:\Manal\Programming\project1\R0_simulation\trace_enveloppe_2m_0.1.pkl", 'rb') as f:
    trace_env = pickle.load(f)

# extract samples from the trace

samples = trace_env.posterior.stack(sample=("chain", "draw"))

# select a random sample from the trace
selected_samples = np.random.choice(range(len(samples['p0'])), size=iter, replace=False)

# extract the values of the parameters
p0_sample = samples['p0'].values[selected_samples]
lambda_sample = samples['lambda'].values[selected_samples]


In [7]:
%%time 

# Perform the simulations

data = []
results = dict()# using dictionary instead of  lists
for k in D: # density
    for n in N: # population size 
            for i in range(iter): # iterations
                p0 = p0_sample[i]
                lambda_ = lambda_sample[i]
                temp_results = []

                for m in range(n): # individuals
                    mu = np.random.uniform(0, 1) # a random number between 0 and 1
                    Rmax= np.sqrt(n/(k * np.pi)) # maximum radius distance
                    r0 = np.sqrt((n * mu)/(k * np.pi)) # generate initial radius distance r0
                    r = r0
                    p = 1 - (p0 * np.exp(-lambda_ * r0)) 
                    proba_minute = p
                    
                    for j in range(time):
                        if j == 0:
                            pass
                        else:
                            theta_ =  np.random.uniform(0, 2*np.pi) # a random number between 0 and pi
                            r = min(np.sqrt(r**2 + d**2 - 2 * r * d * np.cos(theta_) ),Rmax) # generate next radius distance per minute 
                            p = 1 - (p0 * np.exp(-lambda_ * r)) # probability per minute
                            proba_minute *= p # product on all probabilities per minute for each ID
                    
                    temp_results.append(proba_minute)
                    

                #status = [np.random.binomial(T_max, 1 - val) for val in temp_results]
   
                for t in T:
                        status = [np.random.binomial(t, 1 - val) for val in temp_results] # Bernoulli drawing for infectious status (0 or 1) using the final proba
                        a =[k, n, t] # creating the key for the dictionary
                        key = str(a) # writing as a string
                        try:
                            results[key].append(n - status.count(0))
                        except KeyError:
                            results[key]=[n - status.count(0)]
                        

CPU times: total: 3h 25min 13s
Wall time: 3h 28min 44s


In [8]:
results

{'[0.01, 30, 1]': [3,
  1,
  1,
  1,
  2,
  2,
  2,
  3,
  2,
  0,
  2,
  3,
  1,
  3,
  4,
  2,
  0,
  0,
  2,
  0,
  4,
  1,
  3,
  3,
  3,
  6,
  4,
  4,
  2,
  1,
  6,
  2,
  0,
  0,
  1,
  1,
  3,
  1,
  2,
  1,
  5,
  2,
  1,
  4,
  1,
  3,
  1,
  2,
  5,
  0,
  1,
  2,
  2,
  0,
  5,
  1,
  1,
  1,
  1,
  1,
  2,
  0,
  3,
  4,
  2,
  0,
  4,
  0,
  2,
  4,
  6,
  1,
  2,
  2,
  2,
  1,
  0,
  3,
  5,
  1,
  1,
  4,
  3,
  2,
  3,
  6,
  3,
  2,
  3,
  2,
  0,
  3,
  0,
  0,
  7,
  4,
  1,
  1,
  0,
  0],
 '[0.01, 30, 2]': [7,
  2,
  4,
  4,
  3,
  5,
  6,
  2,
  6,
  2,
  3,
  7,
  5,
  4,
  7,
  6,
  5,
  5,
  0,
  5,
  7,
  3,
  4,
  5,
  3,
  6,
  5,
  2,
  5,
  2,
  8,
  7,
  4,
  4,
  4,
  1,
  3,
  3,
  5,
  6,
  7,
  4,
  8,
  5,
  4,
  4,
  0,
  7,
  3,
  4,
  5,
  4,
  4,
  2,
  6,
  4,
  6,
  4,
  1,
  3,
  2,
  4,
  4,
  7,
  4,
  4,
  3,
  4,
  3,
  2,
  11,
  6,
  4,
  2,
  8,
  3,
  2,
  4,
  5,
  3,
  6,
  5,
  2,
  3,
  5,
  8,
  5,
  2,
  4,
  2,
  2,
  6,
  2,

In [9]:
with open(r"D:\Manal\Programming\project1\R0_simulation\results_dictionary_v5.pkl", 'wb') as f:
    pickle.dump(results, f)

In [10]:
triplets = list(results.keys())

In [11]:
col_names = ['Density', 'Population_size', 'Infectious_time'] + [f'iter_{i}' for i in range(1, max(len(value) for value in results.values()) + 1)]


In [12]:
df = pd.DataFrame(
    [[float(triplet[1:-1].split(', ')[0])] + [int(x) for x in triplet[1:-1].split(', ')[1:]] + value for triplet, value in results.items()],
    columns=col_names
)

In [13]:
df

,Density,Population_size,Infectious_time,iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,...,iter_91,iter_92,iter_93,iter_94,iter_95,iter_96,iter_97,iter_98,iter_99,iter_100
0,0.01,30,1,3,1,1,1,2,2,2,...,0,3,0,0,7,4,1,1,0,0
1,0.01,30,2,7,2,4,4,3,5,6,...,2,6,2,5,5,4,1,4,8,2
2,0.01,30,3,10,6,2,8,7,4,1,...,3,10,1,2,10,5,1,8,9,4
3,0.01,30,4,8,6,4,13,7,4,9,...,8,9,5,8,14,5,7,6,4,8
4,0.01,30,5,12,6,10,12,9,3,14,...,5,12,3,8,10,13,8,5,10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,2.00,1000,3,778,738,720,767,691,600,784,...,710,802,352,571,928,661,718,707,694,726
444,2.00,1000,4,854,807,846,842,806,695,876,...,812,877,433,674,963,756,812,801,791,790
445,2.00,1000,5,911,876,872,916,855,775,913,...,873,913,509,751,984,835,859,843,861,868
446,2.00,1000,7,966,936,943,962,930,873,977,...,953,970,641,844,999,900,941,930,937,935


In [14]:
# Save the simulation results into a csv file

df.to_csv("simulation_results_v5.csv", index=False)